In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import shap

In [ ]:
occupancy_rooms=pd.read_csv('imd/barriers_to_housing/occupancy_rating_rooms.csv')

In [ ]:
occupancy_rooms["proportion of overcrowed houses"] = (occupancy_rooms.iloc[:, 8] + occupancy_rooms.iloc[:, 9]) / occupancy_rooms.iloc[:, 4]

In [ ]:
occupancy_rooms.head()

In [ ]:
overcrowded_proportion = occupancy_rooms.iloc[:, [2, 10]]

In [ ]:
central_heating=pd.read_csv('imd/living_env/central_heating.csv')

In [ ]:
central_heating["proportion of no central heating"] = central_heating.iloc[:, 5] / central_heating.iloc[:, 4]

In [ ]:
central_heating_proportion = central_heating.iloc[:, [2, 12]]

In [ ]:
english_proficiency=pd.read_csv('imd/education/english_proficiency.csv')

In [ ]:
english_proficiency["cannot speak english well, or at all"] = (english_proficiency.iloc[:, 8] + english_proficiency.iloc[:, 9]) / english_proficiency.iloc[:, 4]

In [ ]:
unproficient_english_proportion = english_proficiency.iloc[:, [2, 10]]

In [ ]:
highest_qual=pd.read_csv('imd/education/highest_qual.csv')

In [ ]:
highest_qual["level 2 or below"] = (highest_qual.iloc[:, 5] + highest_qual.iloc[:, 6] + highest_qual.iloc[:, 7]) / highest_qual.iloc[:, 4]

In [ ]:
highest_qual["level 3 or below"] = (highest_qual.iloc[:, 5] + highest_qual.iloc[:, 6] + highest_qual.iloc[:, 7] + highest_qual.iloc[:, 8])  / highest_qual.iloc[:, 4]

In [ ]:
highest_qual_proportion = highest_qual.iloc[:, [2, 12, 13]]

In [ ]:
employment=pd.read_csv('imd/employment/employment.csv')

In [ ]:
employment["proportion of households with no adults in employment"] = employment.iloc[:, 5] / employment.iloc[:, 4]

In [ ]:
employment["proportion of households with one person with a long-term health problem or disability"] = employment.iloc[:, 10] / employment.iloc[:, 4]

In [ ]:
employment_proportion = employment.iloc[:, [2, 13, 14]]

In [ ]:
unpaid_care=pd.read_csv('imd/employment/unpaid_care.csv')

In [ ]:
unpaid_care["20 or more hours of unpaid care"] = (unpaid_care.iloc[:, 7] + unpaid_care.iloc[:, 8]) / unpaid_care.iloc[:, 4]

In [ ]:
unpaid_care_proportion = unpaid_care.iloc[:, [2, 9]]

In [ ]:
health=pd.read_csv('imd/health/health.csv')

In [ ]:
health["proportion of bad or very bad health"] = (health.iloc[:, 8] + health.iloc[:, 9]) / health.iloc[:, 4]

In [ ]:
bad_health_proportion = health.iloc[:, [2, 10]]

In [ ]:
long_term_health=pd.read_csv('imd/health/long_term_health.csv')

In [ ]:
long_term_health["day to day activities limited a lot"] = long_term_health.iloc[:, 5] / long_term_health.iloc[:, 4]

In [ ]:
long_term_health_proportion = long_term_health.iloc[:, [2, 8]]

In [ ]:
hours_worked=pd.read_csv('imd/income/hours_worked.csv')

In [ ]:
hours_worked["15 or less hours worked"] = hours_worked.iloc[:, 5] / hours_worked.iloc[:, 4]

In [ ]:
hours_worked_proportion = hours_worked.iloc[:, [2, 19]]

In [ ]:
economic_activity=pd.read_csv('imd/income/economic_activity.csv')

In [ ]:
economic_activity["proportion of unemployed"] = economic_activity.iloc[:, 9] / economic_activity.iloc[:, 3]

In [ ]:
economic_activity_proportion = economic_activity.iloc[:, [2, 21]]

In [ ]:
crime=pd.read_csv('imd/crime/crime.csv')

In [ ]:
crime[crime.iloc[:, 0] == "E01012334"]

In [ ]:
crime["burglary_rate"] = (crime.iloc[:, 4] / crime.iloc[:, 3]) * 1000

In [ ]:
crime["violence_rate"] = (crime.iloc[:, 5] / crime.iloc[:, 3]) * 1000

In [ ]:
crime["cda_rate"] = (crime.iloc[:, 6] / crime.iloc[:, 3]) * 1000

In [ ]:
crime["theft_rate"] = ((crime.iloc[:, 7]+ crime.iloc[:, 8] + crime.iloc[:, 9]) / crime.iloc[:, 3]) * 1000

In [ ]:
crime_rates = crime.iloc[:, [0, 11, 12, 13, 14]]

In [ ]:
scores=pd.read_csv('imd/scores/scores.csv')

In [ ]:
scores_imd = scores.iloc[:, [0, 4]]

In [ ]:
scores_imd.head()

In [ ]:
table_list = [overcrowded_proportion, central_heating_proportion, unproficient_english_proportion, highest_qual_proportion, 
              employment_proportion, unpaid_care_proportion, bad_health_proportion, long_term_health_proportion, hours_worked_proportion, 
              economic_activity_proportion]

In [ ]:
merged = table_list[0]

In [ ]:
for table in table_list[1:]:
    merged = pd.merge(merged, table, on="geography code", how="inner")

In [ ]:
merged = pd.merge(merged, scores_imd, left_on="geography code", right_on = "LSOA code (2011)", how="inner")

In [ ]:
merged = pd.merge(merged, crime_rates, left_on="geography code", right_on = "LSOA", how="inner")

In [ ]:
merged = merged.drop(merged.columns[[13, 15]], axis=1)

In [ ]:
#merged["imd_bin"] = pd.qcut(merged['Index of Multiple Deprivation (IMD) Score'], q=5, labels=False)

In [ ]:
#merged = merged.drop(merged.columns[[13]], axis=1)

In [ ]:
lsoa_column = merged['geography code']

In [ ]:
merged = merged.drop("geography code", axis=1)

In [ ]:
merged.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = merged.drop('Index of Multiple Deprivation (IMD) Score', axis=1)
y = merged['Index of Multiple Deprivation (IMD) Score']

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
print(f"Training data: {len(X_train)}")
print(f"Testing data: {len(X_test)}")

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

gpu = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(gpu)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(gpu)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32).to(gpu)
y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32).to(gpu)

class MLPModel(nn.Module):
    def __init__(self, inputs, hidden, output):
        super(MLPModel, self).__init__()
        self.hidden_layer1 = nn.Linear(inputs, hidden)
        self.relu = nn.ReLU()
        self.hidden_layer2 = nn.Linear(hidden, hidden)
        self.output_layer = nn.Linear(hidden, output)
    
    def forward(self, x):
        x = self.relu(self.hidden_layer1(x))
        x = self.relu(self.hidden_layer2(x))
        x = self.output_layer(x)
        return x

inputs = X_train_tensor.shape[1]
hidden = 64
output = 1

model = MLPModel(inputs, hidden, output).to(gpu)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

epochs = 1000
batch_size = 64

for epoch in range(epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        X_batch = X_train_tensor[i:i+batch_size]
        y_batch = y_train_tensor[i:i+batch_size]
        
        optimizer.zero_grad()
        predictions = model(X_batch)
        loss = criterion(predictions, y_batch.view(-1, 1))
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        print("Epoch", epoch + 1, "/", epochs, "Loss:", round(loss.item(), 4))

In [ ]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)

test_loss = criterion(y_pred, y_test_tensor.view(-1, 1))
print("Test Loss:", round(test_loss.item(), 4))

In [ ]:
y_pred = y_pred.cpu().numpy()
y_test = y_test_tensor.cpu().numpy()
print("Pred:", y_pred[:10])
print("True Val:",y_test_tensor[:10])

In [ ]:
y_train_original = scaler_y.inverse_transform(y_train_scaled.reshape(-1, 1))
y_test_original = scaler_y.inverse_transform(y_test_scaled.reshape(-1, 1))
y_pred_original = scaler_y.inverse_transform(y_pred.reshape(-1, 1))

print("Pred:", y_pred_original[:10])
print("True Val:", y_test_original[:10])

In [ ]:
true_values_bins = pd.qcut(y_test_original.flatten(), 5, labels=False)
pred_values_bins = pd.qcut(y_pred_original.flatten(), 5, labels=False)

print("True Val:", true_values_bins[:10])
print("Pred:", pred_values_bins[:10])

In [ ]:
accuracy = (true_values_bins == pred_values_bins).mean()
print("Accuracy:", round(accuracy, 4))

In [ ]:
X_test_sample = X_test_tensor[:1000].cpu().numpy()

explainer = shap.GradientExplainer(model, X_train_tensor[:1000])

shap_values = explainer.shap_values(X_test_tensor[:1000])
shap_values = shap_values.squeeze(-1)
shap.summary_plot(shap_values, X_test_sample, feature_names=X.columns)

In [ ]:
X_test_sample.shape

In [ ]:
X_test_tensor.shape

In [ ]:
shap_values.shape

In [ ]:
merged.head()

In [ ]:
X_test_tensor[:1000]

In [ ]:
merged.head()

In [ ]:
merged.info()

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Load LSOA boundary GeoJSON
lsoa_map = gpd.read_file("lsoa.geojson")  # Update path if needed

# Filter out Welsh LSOAs
lsoa_map_england = lsoa_map[lsoa_map['LSOA11CD'].str.startswith('E')]

# Rename LSOA ID column to match
scores_imd.rename(columns={'LSOA code (2011)': 'LSOA11CD'}, inplace=True)

# Merge boundaries with deprivation scores
lsoa_map_england = lsoa_map.merge(scores_imd, on="LSOA11CD", how="left")

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
lsoa_map_england.plot(column="Index of Multiple Deprivation (IMD) Score", cmap="OrRd", legend=True, ax=ax)

# Add title
ax.set_title("LSOA Deprivation Scores in England", fontsize=12)

plt.show()

In [ ]:
print(lsoa_map.head())

In [ ]:
print(scores_imd.columns)

In [ ]:
print(lsoa_map.columns) 

In [ ]:
import folium

lsoa_map_england = lsoa_map_england.to_crs(epsg=4326)

# centre on England
m = folium.Map(location=[52.3555, -1.1743], zoom_start=6)

folium.Choropleth(
    geo_data=lsoa_map_england,
    data=scores_imd,
    columns=["LSOA11CD", "Index of Multiple Deprivation (IMD) Score"],
    key_on="feature.properties.LSOA11CD",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Index of Multiple Deprivation (IMD) Score"
).add_to(m)

# Save
m.save("england_deprivation_map.html")
m

In [ ]:
merged.head()

In [ ]:
# Evaluation mode
model.eval()

# Standardize dataset
X_full = merged.drop(columns=['Index of Multiple Deprivation (IMD) Score'])
X_full_scaled = scaler_X.transform(X_full)

# Convert to PyTorch tensor and move to GPU  
X_full_tensor = torch.tensor(X_full_scaled, dtype=torch.float32).to(gpu)

# Make prediction using trained model
with torch.no_grad():
    y_pred_scaled = model(X_full_tensor)

# Convert predictions back to original scale
y_pred = scaler_y.inverse_transform(y_pred_scaled.cpu().numpy())

# Create DataFrame with LSOA and Predicted IMD Score
results_df = pd.DataFrame({
    'LSOA': lsoa_column, 
    'Predicted IMD Score': y_pred.flatten()
})

# Save to CSV
results_df.to_csv('predicted_imd_scores.csv', index=False)

In [ ]:
import folium
from folium.plugins import Fullscreen
# Load IMD 2011 predictions
results_df = pd.read_csv("predicted_imd_scores.csv")

# Load LSOA GeoJSON file
gdf = gpd.read_file("lsoa.geojson")

# Match column name
gdf = gdf.rename(columns={'LSOA11CD': 'LSOA'})  
merged_gdf = gdf.merge(results_df, on="LSOA", how="left")

# Centre of england
m = folium.Map(location=[52.3555, -1.1743], zoom_start=6, tiles="cartodbpositron")

folium.Choropleth(
    geo_data=merged_gdf,
    name="IMD Score",
    data=merged_gdf,
    columns=["LSOA", "Predicted IMD Score"],
    key_on="feature.properties.LSOA",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Predicted IMD Score"
).add_to(m)

Fullscreen().add_to(m)

m.save("imd_map.html")
m

**2021 Census**

In [ ]:
occupancy_rooms2=pd.read_csv('imd2021/barriers_to_housing/occupancy_rating_rooms2.csv')

In [ ]:
occupancy_rooms2["proportion of overcrowed houses"] = (occupancy_rooms2.iloc[:, 5] + occupancy_rooms2.iloc[:, 6]) / occupancy_rooms2.iloc[:, 1]

In [ ]:
overcrowded_proportion2 = occupancy_rooms2.iloc[:, [0, 7]]

In [ ]:
central_heating2=pd.read_csv('imd2021/living_env/central_heating2.csv')

In [ ]:
central_heating2["proportion of no central heating"] = central_heating2.iloc[:, 2] / central_heating2.iloc[:, 1]

In [ ]:
central_heating_proportion2 = central_heating2.iloc[:, [0, 3]]

In [ ]:
english_proficiency2=pd.read_csv('imd2021/education/english_proficiency2.csv')

In [ ]:
english_proficiency2["cannot speak english well, or at all"] = (english_proficiency2.iloc[:, 2] + english_proficiency2.iloc[:, 3]) / english_proficiency2.iloc[:, 1]

In [ ]:
unproficient_english_proportion2 = english_proficiency2.iloc[:, [0, 4]]

In [ ]:
highest_qual2=pd.read_csv('imd2021/education/highest_qual2.csv')

In [ ]:
highest_qual2["level 2 or below"] = (highest_qual2.iloc[:, 2] + highest_qual2.iloc[:, 3] + highest_qual2.iloc[:, 4]) / highest_qual2.iloc[:, 1]

In [ ]:
highest_qual2["level 3 or below"] = (highest_qual2.iloc[:, 2] + highest_qual2.iloc[:, 3] + highest_qual2.iloc[:, 4] + highest_qual2.iloc[:, 6])  / highest_qual2.iloc[:, 1]

In [ ]:
highest_qual_proportion2 = highest_qual2.iloc[:, [0, 7, 8]]

In [ ]:
employment3=pd.read_csv('imd2021/employment/employment3.csv')

In [ ]:
employment3["proportion of households with no adults in employment"] = (employment3.iloc[:, 2] + employment3.iloc[:, 3]) / employment3.iloc[:, 1]

In [ ]:
employment_proportion3 = employment3.iloc[:, [0, 4]]

In [ ]:
employment2=pd.read_csv('imd2021/employment/employment4.csv')

In [ ]:
employment2["proportion of households with one person with a long-term health problem or disability"] = employment2.iloc[:, 2] / employment2.iloc[:, 1]

In [ ]:
employment_proportion2 = employment2.iloc[:, [0, 4]]

In [ ]:
employment_proportion2.head()

In [ ]:
unpaid_care2=pd.read_csv('imd2021/employment/unpaid_care2.csv')

In [ ]:
unpaid_care2["20 or more hours of unpaid care"] = (unpaid_care2.iloc[:, 2] + unpaid_care2.iloc[:, 3]) / unpaid_care2.iloc[:, 1]

In [ ]:
unpaid_care_proportion2 = unpaid_care2.iloc[:, [0, 4]]

In [ ]:
health2=pd.read_csv('imd2021/health/health2.csv')

In [ ]:
health2["proportion of bad or very bad health"] = (health2.iloc[:, 2] + health2.iloc[:, 3]) / health2.iloc[:, 1]

In [ ]:
bad_health_proportion2 = health2.iloc[:, [0, 4]]

In [ ]:
long_term_health2=pd.read_csv('imd2021/health/long_term_health2.csv')

In [ ]:
long_term_health2["day to day activities limited a lot"] = long_term_health2.iloc[:, 3] / long_term_health2.iloc[:, 1]

In [ ]:
long_term_health_proportion2 = long_term_health2.iloc[:, [0, 4]]

In [ ]:
hours_worked2=pd.read_csv('imd2021/income/hours_worked2.csv')

In [ ]:
hours_worked2["15 or less hours worked"] = hours_worked2.iloc[:, 2] / hours_worked2.iloc[:, 1]

In [ ]:
hours_worked_proportion2 = hours_worked2.iloc[:, [0, 3]]

In [ ]:
economic_activity2=pd.read_csv('imd2021/income/economic_activity2.csv')

In [ ]:
economic_activity2["proportion of unemployed"] = economic_activity2.iloc[:, 2] / economic_activity2.iloc[:, 1]

In [ ]:
economic_activity_proportion2 = economic_activity2.iloc[:, [0, 4]]

In [ ]:
crime2=pd.read_csv('imd2021/crime/crime2.csv')

In [ ]:
crime2["burglary_rate"] = (crime2.iloc[:, 4] / crime2.iloc[:, 3]) * 1000

In [ ]:
crime2["violence_rate"] = (crime2.iloc[:, 5] / crime2.iloc[:, 3]) * 1000

In [ ]:
crime2["cda_rate"] = (crime2.iloc[:, 6] / crime2.iloc[:, 3]) * 1000

In [ ]:
crime2["theft_rate"] = ((crime2.iloc[:, 7]+ crime2.iloc[:, 8] + crime2.iloc[:, 9]) / crime2.iloc[:, 3]) * 1000

In [ ]:
crime_rates2 = crime2.iloc[:, [0, 11, 12, 13, 14]]

In [ ]:
crime_rates2.head()

In [ ]:
crime2.info()

In [ ]:
table_list2 = [overcrowded_proportion2, central_heating_proportion2, unproficient_english_proportion2, highest_qual_proportion2, employment_proportion3, 
              employment_proportion2, unpaid_care_proportion2, bad_health_proportion2, long_term_health_proportion2, hours_worked_proportion2, 
              economic_activity_proportion2]

In [ ]:
merged2 = table_list2[0]

In [ ]:
for table in table_list2[1:]:
    merged2 = pd.merge(merged2, table, on="2021 super output area - lower layer", how="inner")

In [ ]:
merged2.head()

In [ ]:
merged2.info()

In [ ]:
merged2["2021 super output area - lower layer"] = merged2["2021 super output area - lower layer"].str.split(" : ").str[0]

In [ ]:
merged2.head()

In [ ]:
merged2 = pd.merge(merged2, crime_rates2, left_on="2021 super output area - lower layer", right_on = "LSOA", how="inner")

In [ ]:
merged2.head()

In [ ]:
merged2.info()

In [ ]:
lsoa_column2021 = merged2["LSOA"]
merged2 = merged2.drop(merged2.columns[[0, 13]], axis=1)

X_merged2 = merged2.copy()  

# Standardize features
X_merged2_scaled = scaler_X.transform(X_merged2)

# Convert to PyTorch Tensor
X_merged2_tensor = torch.tensor(X_merged2_scaled, dtype=torch.float32).to(gpu)

# Generate Predictions
model.eval()
with torch.no_grad():
    predictions = model(X_merged2_tensor)

# Convert Predictions Back to Original Scale
predictions_np = scaler_y.inverse_transform(predictions.cpu().numpy())

# Create results DataFrame
results_df2021 = pd.DataFrame({
    'LSOA': lsoa_column2021, 
    'Predicted IMD Score': predictions_np.flatten()
})

In [ ]:
merged2.head()

In [ ]:
merged.head()

In [ ]:
results_df2021.head()

In [ ]:
# Load LSOA Shapefile
gdf2021 = gpd.read_file("imd2021/lsoa2021.zip")

# match column names
gdf2021 = gdf.rename(columns={'LSOA11CD': 'LSOA'})
merged_gdf2021 = gdf.merge(results_df2021, on="LSOA", how="left")

# Centre
m = folium.Map(location=[52.3555, -1.1743], zoom_start=6, tiles="cartodbpositron")

folium.Choropleth(
    geo_data=merged_gdf2021,
    name="IMD Score",
    data=merged_gdf2021,
    columns=["LSOA", "Predicted IMD Score"],
    key_on="feature.properties.LSOA",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Predicted IMD Score"
).add_to(m)

Fullscreen().add_to(m)

m.save("imd_map.html")
m

In [ ]:
results_df.head()

In [ ]:
results_df2021.head()

In [ ]:
merged_results_df = pd.merge(results_df, results_df2021, on="LSOA", how="inner")

In [ ]:
merged_results_df.head()

In [ ]:
# Calculate the change in IMD scores
merged_results_df['IMD_change'] = merged_results_df['Predicted IMD Score_y'] - merged_results_df['Predicted IMD Score_x']

# Merge the GeoDataFrame with the merged_results_df based on LSOA
lsoa_map_england = lsoa_map_england.rename(columns={'LSOA11CD': 'LSOA'})
merged_change_gdf = lsoa_map_england.merge(merged_results_df, on="LSOA", how="left")

# centre map
m = folium.Map(location=[52.3555, -1.1743], zoom_start=6, tiles="cartodbpositron")

folium.Choropleth(
    geo_data=merged_change_gdf,
    name="IMD Change",
    data=merged_change_gdf,
    columns=["LSOA", "IMD_change"],
    key_on="feature.properties.LSOA",
    fill_color="RdYlBu_r",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Change in IMD Score (2021 - 2011)",
).add_to(m)

Fullscreen().add_to(m)

m.save("imd_change_map.html")

m

In [ ]:
merged.head()

In [ ]:
merged.describe()

In [ ]:
merged2.describe()

In [ ]:
merged_results_df.describe()

In [ ]:
merged_results_df = merged_results_df.rename(columns = {'Predicted IMD Score_x': 'IMD predictions using 2011 Census', 'Predicted IMD Score_y': 'IMD predictions using 2021 Census'})

In [ ]:
merged_results_df.describe()